In [2]:
from lxml import html
import requests
from xml.etree import ElementTree




host = 'https://www.hkdentists.com.hk'
url = 'https://www.hkdentists.com.hk/index.php?actiontype=search&dentist_name=&clinic_dist=showall&practice=showall&specialist=&openhour=&language=chi'
page = requests.get(url)
tree = html.fromstring(page.content)



In [3]:
clinics = tree.xpath('//a[@class="link_text"]')

In [5]:
#body/table/tr/td[2]/table/tr[3]/td/table
xpath = '/html/body/table'
records = []
for clinic in clinics:
    clinic_url = host + clinic.attrib['href'][1:] + '&language=chi'
    #print(clinic_url)
    clinic_page = requests.get(clinic_url)
    
    clinic_page_tree = html.fromstring(clinic_page.content)
    clinic_page_info = clinic_page_tree.find('body/table/tr/td[2]/table/tr[3]/td/table').getchildren()
    #print(clinic_page_info)
    record = {}
    for row in clinic_page_info:
        if len(row) != 2:
            continue
        k = '-'.join([x.strip() for x in row[0].itertext()])
        v = '-'.join([x.strip() for x in row[1].itertext()])
        #print(k, v)
        record[k] = v
    records.append(record)
    #break
print(len(records))

1071


In [21]:
print(records[1])

{'Dentist Name:': 'AU HUNG ON, ADAM', '牙醫姓名:': '區雄安', '業務類型:': '私家', '專科:': '兒童齒科', '資歷': '香港大學牙醫學士-英國愛丁堡皇家外科醫學院兒童齒科學院員-香港牙科醫學院院士(兒童齒科)專科-香港醫學專科學院院士(牙科)-澳紐皇家牙科醫學院院士', '診所地址:': '中環皇後大道中36號-興瑋大廈1102室', '地區:': '中西區', '電話號碼:': '25225044', '傳真號碼:': '', '星期一:': '09:30 am - 1:30  pm , \xa0 \xa0 3:00  pm - 7:00  pm', '星期二:': '09:30 am - 1:30  pm , \xa0 \xa0 3:00  pm - 7:00  pm', '星期三:': '09:30 am - 1:30  pm , \xa0 \xa0 3:00  pm - 7:00  pm', '星期四:': '09:30 am - 1:30  pm , \xa0 \xa0 3:00  pm - 7:00  pm', '星期五:': '09:30 am - 1:30  pm , \xa0 \xa0 3:00  pm - 7:00  pm', '星期六:': '09:30 am - 1:30  pm , \xa0 \xa0 3:00  pm - 7:00  pm', '星期日:': '休息', '公眾假期:': '休息', '能否提供緊急服務:': '能', '緊急聯絡電話:': '', '電郵地址:': 'adam@adamantdental.com'}


In [25]:
import csv

with open('output.csv', 'w', encoding='utf-8', newline='') as f:  # Just use 'w' mode in 3.x
    w = csv.DictWriter(f, records[1].keys(), extrasaction='ignore')
    w.writeheader()
    for x in records:
        w.writerow(x)